In [1]:
import numpy as np
import pandas as pd
import gc

In [2]:
# Asking user's inputs
# Sample responses:
# user_vcf_file_path = /content/WES_JH_TM_full_vep.vcf
# first_letter_unwanted_sample = T
# batch_size = 20
# LD_threshold = 0.2
# user_data_population = JH

# Step 1
user_vcf_file_path = str(input("(1) What is the path of the VCF file containing your exome data? "))
print("- Path of VCF file containing user's exome data:", user_vcf_file_path)
print("")

first_letter_unwanted_sample = str(input("(2) What is the first letter of the sample IDs for populations that you want to exclude from the analysis? "))
print("- First letter of the sample IDs for populations that user wants to exclude from the analysis:", first_letter_unwanted_sample)
print("")

# Step 2
batch_size = int(input("(3) What batch size would you like to use when processing the 1000 Genome data? (< 20 is recommended) "))
print("- Batch size for 1000 Genome data processing:", batch_size)
print("")

# Step 3
LD_threshold = float(input("(4) What is the threshold value for Linkage Disequilibrium (LD)? (0.2 is recommended) "))
print("- Threshold value for Linkage Disequilibrium (LD):", LD_threshold)
print("")

# Step 4
user_data_population = str(input("(5) What abbreviation would you like to use for the population concerning your exome data? "))
print("- Selected abbreviation for the population of user's exome data:", user_data_population)
print("")

(1) What is the path of the VCF file containing your exome data? /content/WES_JH_TM_full_vep.vcf
- Path of VCF file containing user's exome data: /content/WES_JH_TM_full_vep.vcf

(2) What is the first letter of the sample IDs for populations that you want to exclude from the analysis? T
- First letter of the sample IDs for populations that user wants to exclude from the analysis: T

(3) What batch size would you like to use when processing the 1000 Genome data? (< 20 is recommended) 20
- Batch size for 1000 Genome data processing: 20

(4) What is the threshold value for Linkage Disequilibrium (LD)? (0.2 is recommended) 0.2
- Threshold value for Linkage Disequilibrium (LD): 0.2

(5) What abbreviation would you like to use for the population concerning your exome data? JH
- Selected abbreviation for the population of user's exome data: JH



In [ ]:
import numpy as np
import pandas as pd
import gc

# Step 1: Extracting and processing user's exome VCF data
from stepOne_user_file_processor import user_file_processor

print("Step 1:")
df2 = user_file_processor(user_vcf_file_path, first_letter_unwanted_sample)
gc.collect()

# Step 2: Extracting SNPs from 1000 Genome data
from stepTwo_Thousand_Genome_SNP_extractor_v3 import Thousand_Genome_SNP_extractor


print("Step 2:")
# Specifying path for csv file of user's processed exome data
#user_file_path = "/content/User_Exome_Analysis.csv"
user_file_path = "/content/Jehai_Exome_Analysis_Step4_v2.csv"

# Specifying the list of chromosomes to be examined
chr_num_list = []

for chr_num in range(1, 23):
  chr_num_list.append(chr_num)

  if chr_num == 22:
    chr_num_list = list(map(str, chr_num_list))
    chr_num_list.append("X")

# Storing the title of csv files containing extracted SNPs for each chromosome in a list
SNP_extracted_list = []

for chr in chr_num_list:
  gc.collect()
  csv_title = Thousand_Genome_SNP_extractor(chr, user_file_path, batch_size)
  SNP_extracted_list.append(csv_title)

# Step 3: Processing data of SNPs from 1000 Genome data
from stepThree_Thousand_Genome_data_processor import Thousand_Genome_data_processor

print("Step 3:")

ID_to_population_file_list = ["/content/AFR_373509-SampleGenotypes-Homo_sapiens_Variation_Sample_rs1143634.csv",
             "/content/AMR_373510-SampleGenotypes-Homo_sapiens_Variation_Sample_rs1143634.csv",
             "/content/EAS_373518-SampleGenotypes-Homo_sapiens_Variation_Sample_rs1143634.csv",
             "/content/EUR_373520-SampleGenotypes-Homo_sapiens_Variation_Sample_rs1143634.csv",
             "/content/SAS_373535-SampleGenotypes-Homo_sapiens_Variation_Sample_rs1143634.csv"]

df_10_list = Thousand_Genome_data_processor(SNP_extracted_list, ID_to_population_file_list, LD_threshold)
gc.collect()

# Step 4: Compiling SNPs from 1000 Genome data and user's exome data
from stepFour_Thousand_Genome_data_compiler import Thousand_Genome_data_compiler

print("Step 4:")

user_data_df = pd.read_csv(user_file_path)

df_final_transposed = Thousand_Genome_data_compiler(user_file_path, df_10_list, user_data_population)
del df_10_list
gc.collect()

# Step 5: Conducting 2D and 3D PCA on the compiled data
from stepFive_Thousand_Genome_PCA import Thousand_Genome_PCA

Thousand_Genome_PCA(df_final_transposed, user_data_population)

Step 1:
Before removing unnecessary samples:  77820 SNPs and 79 samples.
After removing unnecessary samples:  77820 SNPs and 34 samples.
After removing INDELS:  75176 SNPs.
- Genotypes have been encoded as numerical values successfully!
